<a href="https://colab.research.google.com/github/hslog16/hackathon/blob/master/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

In [0]:
!ls

datalab  nltk_data  res.csv  test.csv  train.csv


In [0]:
!mv 0000000000002747_test_twitter_x_test.csv test.csv

In [0]:
!mv 0000000000002747_training_twitter_x_y_train.csv  train.csv

In [0]:
import pandas as pd
df = pd.read_csv("train.csv")

In [0]:
y = pd.DataFrame()

In [0]:
y['input'] = df['text']

In [0]:
y['out'] = df['airline_sentiment']

In [0]:
from nltk.tokenize import sent_tokenize , word_tokenize
import numpy as np

In [0]:
from random import shuffle

In [0]:
sentiment = np.array(y)

In [0]:
senti = [(word_tokenize(word) , cat) for word , cat in sentiment]

In [0]:
shuffle(senti)

In [0]:
from nltk import pos_tag

In [0]:
from nltk.corpus import wordnet

In [0]:
def simple_pos(word):
  if word.startswith('J'):
    return wordnet.ADJ
  elif word.startswith('V'):
    return wordnet.VERB
  elif word.startswith('N'):
    return wordnet.NOUN
  elif word.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [0]:
from nltk.corpus import stopwords
import string
stop = list(set(stopwords.words('english')))
stop += list(string.punctuation)

In [0]:
from nltk.stem import WordNetLemmatizer as lemme
lemma = lemme()

In [0]:
def clean(words):
  output = []
  for w in words:
    if w.lower() not in stop:
      pos = pos_tag([w])
      clean_word = lemma.lemmatize(w , pos = simple_pos(pos[0][1]))
      output.append(clean_word.lower())
  return output

In [0]:
finale = [ (clean(docs),cat)  for docs , cat in senti ]

In [0]:
shuffle(finale)

In [0]:
train = finale

In [0]:
words=[]
for w in train:
  words+=w[0]

In [0]:
import nltk
init_features = nltk.FreqDist(words)

In [0]:
init_features.keys()

dict_keys(['usairways', 'stuck', 'ramp', 'dca', 'us', 'air', 'computer', 'system', 'crashed', '...', 'everywhere', 'united', 'would', 'like', '2', 'speak', 'vp', 'custexp', 'jimmy', 'samartzis', 'sent', 'survey/email', 'awful', 'flight', 'exp', 'http', '//t.co/otfz7cyguq', 'scm1133', 'hate', 'delay', 'try', '//t.co/7stktjxan1', 'although', "'re", 'sure', 'timbit', 'southwestair', 'wife', 'need', 'help', 'strand', 'chicago', 'ca', "n't", 'get', 'monday', 'wo', 'find', 'bag', 'volume', 'high', 'frustrate', 'checked', 'logan', 'give', 'luggage', 'ticket', 'lose', 'amp', 'ownership', 'jetblue', 'june', 'yet', '😊', 'americanair', 'thank', 'bumping', 'people', 'ten', 'minute', 'takeoff', 'overbooked', 'fail', 'delayed', 'cabcelled', 'virginamerica', 'watch', 'best', 'student', 'film', 'country', '35,000', 'foot', 'cmfat35000feet', '//t.co/kek5pdmgif', 'possible', 'call', 'swa', 'phone', 'put', 'hold', 'hour', 'cut', 'dmed', 'detail', 'cool', 'announce', '``', "''", 'everyone', 'check', '4', 

In [0]:
feature = []

In [0]:
for i in init_features.keys():
  #if(len(i)>2):
    feature.append( (i , init_features[i]) )

In [0]:
from operator import itemgetter
feature.sort(key = itemgetter(1),reverse=True)

In [0]:
feature = feature[:7100]

In [0]:
features = [ f[0] for f in feature]

In [0]:
len(features)

7100

In [0]:
def make_dict(words):
  current_feature = {}
  word = set(words)
  for w in features:
    current_feature[w] = w in word
  return current_feature

In [0]:
train_fin = [ (make_dict(doc) , cat)  for doc , cat in train]

In [0]:
test_fin = [ (make_dict(doc) , cat)  for doc , cat in test]

In [0]:
temp_fin = [ make_dict(doc)  for doc , cat in test]

In [0]:
from nltk import NaiveBayesClassifier as nb

In [0]:
classifier = nb.train(train_fin)

In [0]:
nltk.classify.accuracy(classifier , test_fin)

0.7712204007285974

In [0]:
np.savetxt('res.csv',res,fmt='%s')

2745

In [0]:
classifier.show_most_informative_features(15)

Most Informative Features
                   kudos = True           positi : negati =     51.9 : 1.0
               fantastic = True           positi : negati =     34.2 : 1.0
                favorite = True           positi : negati =     34.2 : 1.0
             outstanding = True           positi : negati =     29.1 : 1.0
                 helpful = True           positi : neutra =     25.4 : 1.0
                   flyfi = True           positi : negati =     24.1 : 1.0
                   thank = True           positi : negati =     23.0 : 1.0
                 awesome = True           positi : negati =     22.6 : 1.0
                    hold = True           negati : positi =     22.3 : 1.0
               beautiful = True           positi : negati =     21.5 : 1.0
                      hr = True           negati : positi =     21.4 : 1.0
                   daily = True           neutra : negati =     20.6 : 1.0
                  street = True           neutra : negati =     20.6 : 1.0

In [0]:
df_test = pd.read_csv("test.csv")
y_test = pd.DataFrame()

In [0]:
y_test['input'] = df_test['text']

In [0]:
sentiment = np.array(y_test)

In [0]:
senti_test = [ (word_tokenize(word[0])) for word in sentiment]

In [0]:
finale = [ clean(docs)  for docs in senti_test ]

In [0]:
test_fin = [ make_dict(doc)  for doc in finale]

In [0]:
res=[]

In [0]:
x=0
for i in test_fin:
  x+=1
  if(x%100==0):
    print(x)
  res.append(classifier.classify(i))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600


In [0]:
np.savetxt('res.csv',res,fmt='%s')

In [0]:
!ls

datalab  nltk_data  res.csv  test.csv  train.csv


In [0]:
from google.colab import files
files.download('res.csv')

In [0]:
from nltk.classify.scikitlearn import SklearnClassifier as sc

In [0]:
from sklearn.svm import SVC

In [0]:
svc = SVC()
clf_svm = sc(svc)

In [0]:
clf_svm.train(train_fin)

<SklearnClassifier(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))>

In [0]:
clf_svm.classify(test_fin[0])

'negative'

In [0]:
res=[]
x=0
for i in test_fin:
  x+=1
  if(x%100==0):
    print(x)
  res.append(clf_svm.classify(i))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600


In [0]:
np.savetxt('res_svm.csv',res,fmt='%s')
from google.colab import files
files.download('res_svm.csv')

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rfc = RandomForestClassifier(verbose = 4 , warm_start=True)
clf_random = sc(rfc)

In [0]:
clf_random.train(train_fin)

building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s


building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.6s finished


<SklearnClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=4, warm_start=True))>

In [0]:
res=[]
x=0
for i in test_fin:
  x+=1
  if(x%100==0):
    print(x)
  res.append(clf_random.classify(i))

In [0]:
np.savetxt('res_random.csv',res,fmt='%s')
from google.colab import files
files.download('res_random.csv')

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /content/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True